In [1]:
%config IPCompleter.greedy=True

In [11]:
import pandas as pd
import numpy as np
rates = pd.read_excel('../O-U Simulation.xlsx', sheet_name="Sheet1")
rates.set_index('Year', inplace=True)
rates.columns


Index(['Carbon Emission(X_t)', 'X(t+∆t)'], dtype='object')

$B =(X^TX)^{-1}X^Ty$ 

In [12]:
rates['X(t+∆t)'] = rates['Carbon Emission(X_t)'].shift(-1)
rates = rates.iloc[:-1,:]
rates.loc[:,'constant'] = 1
X = rates.loc[:,['constant', 'Carbon Emission(X_t)']].values
X_T  = np.transpose(X)
X_inv = (X)
Beta = np.linalg.inv(X_T.dot(X)).dot(X_T).dot(rates['X(t+∆t)'].values) 
intercept, coefficient = Beta

$\sigma = \sqrt{\frac{e^Te}{n-p}}$

In [13]:
y_hat = X.dot(Beta)
e = y_hat-rates['X(t+∆t)'].values
std_err =np.sqrt( np.transpose(e).dot(e)/(len(rates)-2))

$$C_{t+\Delta_t}=C_te^{-\alpha\Delta_t} + b(1-e^{-\alpha\Delta_t}) + \epsilon_t$$
$$Var(\epsilon_t) = \frac{\sigma^2(1-e^{-2\alpha\Delta_t})}{2a}$$

In [14]:
alpha = -np.log(coefficient)

b = intercept/(1-np.exp(-alpha))

sigma = np.sqrt(((std_err**2)*2*alpha)/(1-np.exp(-2*alpha)))



In [15]:


last_observation = rates.iloc[-1,:]['X(t+∆t)']
last_observation

0.30592372189616096

In [16]:
from dataclasses import dataclass

@dataclass
class SimulationsGen:

    pull_back: float
    long_term_average:float
    deltaT:float
    sigma:float
    init_val:float
    n:int
    sim_count:int
    results: list
    
        
    def _get_delta_c(self, previous_val ):
        delta_C = self.pull_back*(self.long_term_average-previous_val)*self.deltaT + \
                sigma*np.sqrt(self.deltaT)*np.random.standard_normal((1,))[0]
        return delta_C   


    def _generate_one_simulation(self):
        simulation = np.empty(self.n)
        simulation[0] = self.init_val
        for i in range(1,self.n):
            simulation[i] = simulation[i-1] + self._get_delta_c(simulation[i-1])
        return simulation
    
    def generate_many_simulations(self):
        self.results = [self._generate_one_simulation() for i in range(self.sim_count)]
        
        
    
    

In [17]:
gen = SimulationsGen(pull_back=alpha, long_term_average=b, 
          deltaT=1, sigma=sigma, init_val=last_observation, n=400, sim_count=1000, results=[])

gen.generate_many_simulations()


In [18]:
pd.DataFrame(gen.results).T

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.305924,0.305924,0.305924,0.305924,0.305924,0.305924,0.305924,0.305924,0.305924,0.305924,...,0.305924,0.305924,0.305924,0.305924,0.305924,0.305924,0.305924,0.305924,0.305924,0.305924
1,0.347048,0.250574,0.337108,0.274526,0.321825,0.289344,0.271481,0.311846,0.306222,0.292611,...,0.270184,0.313019,0.340880,0.286163,0.243454,0.303490,0.316370,0.271974,0.301813,0.339407
2,0.330530,0.181924,0.323188,0.263043,0.257158,0.217018,0.218613,0.291114,0.317571,0.311110,...,0.309455,0.214033,0.346819,0.324329,0.203358,0.274284,0.320947,0.273844,0.250192,0.300858
3,0.282115,0.215901,0.309744,0.265099,0.296631,0.261916,0.166348,0.334487,0.325391,0.331696,...,0.266542,0.245318,0.329120,0.272123,0.227842,0.359342,0.311866,0.328807,0.188801,0.302266
4,0.281248,0.196571,0.362091,0.204675,0.291737,0.322052,0.236646,0.314272,0.319579,0.333766,...,0.264530,0.261366,0.272649,0.324735,0.266955,0.337820,0.342983,0.241446,0.162325,0.291573
5,0.280095,0.270612,0.354180,0.221176,0.329816,0.321287,0.213978,0.317381,0.280469,0.281759,...,0.326567,0.226289,0.301194,0.317732,0.290253,0.360199,0.366120,0.261352,0.216565,0.279532
6,0.267063,0.284327,0.355711,0.235848,0.331560,0.346856,0.172617,0.307140,0.222630,0.291655,...,0.305691,0.237114,0.286304,0.325439,0.287272,0.337129,0.334507,0.200023,0.230911,0.230335
7,0.260994,0.220230,0.338263,0.233862,0.273443,0.349100,0.224004,0.296680,0.236626,0.271354,...,0.310537,0.288044,0.217229,0.306463,0.304528,0.315246,0.324068,0.187484,0.193138,0.170227
8,0.368058,0.227392,0.361814,0.274663,0.248155,0.360217,0.271633,0.221852,0.267801,0.247651,...,0.324754,0.200403,0.245795,0.291677,0.233526,0.266152,0.302894,0.265543,0.238773,0.164880
9,0.382938,0.246375,0.305781,0.286401,0.259934,0.409082,0.309308,0.229342,0.257948,0.222064,...,0.278579,0.206060,0.290862,0.240279,0.175406,0.308827,0.301581,0.300201,0.219211,0.179086
